In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Player(object): # 플레이어 클래스 정의
    def __init__(self, currentSum, usableAce, dealersCard):
      # currentSum: 플레이어 카드의 현재 합
      # dealersCard: 딜러의 카드
      # usableAce: 에이스가 11로 사용 가능한지의 여부
      # usingAce: 현재 에이스를 11로 사용중인지의 여부
        self.currentSum = currentSum
        self.dealersCard = dealersCard
        self.usableAce = usableAce
        self.usingAce = self.usableAce

    def ReceiveCard(self, card):
        if self.usingAce and self.currentSum + card > 21:  # 만약 플레이어가 에이스를 사용중이고 현재 카드의 합과 새 카드의 합이 21이 넘으면
            self.usingAce = False # 에이스 사용 여부를 False로 바꾸고
            self.currentSum += card - 10 # 현재 카드 합에서 10를 빼기 (에이스를 11 -> 1로 바꿨으니깐)
        else: # 21을 넘지 않는다면
            self.currentSum += card # 새 카드를 현재 카드 합에 더하기

    def GetState(self):# 현재 플레이어가 관측하는 상태
        return (self.currentSum, self.usableAce, self.dealersCard)

    def GetValue(self):# 현재 합
        return self.currentSum

    def ShouldHit(self, policy):# 현재 상태에서 정책이 Hit을 권하는지 권하지 않는지
        return policy[self.GetState()]

    def Bust(self): # 버스트 여부 판단
        return self.GetValue() > 21

class Dealer(object): # 딜러 클래스 정의
    def __init__(self, cards): # 딜러의 카드
        self.cards = cards

    def ReceiveCard(self, card): # 새 카드 추가
        self.cards.append(card)

    def GetValue(self): # 합 계산
        currentSum = 0
        aceCount = 0

        for card in self.cards:
            if card == 1: # 만약 에이스가 나오면
                aceCount += 1 # 에이스 카운트 증가
            else:
                currentSum += card

        while aceCount > 0: # 만약 에이스가 있으면
            aceCount -= 1 # 에이스 카운트 감소
            currentSum += 11 # 현재 합에 11 추가

            if currentSum > 21: # 현재 합이 21이 넘으면
                aceCount += 1 # 에이스 카운트 증가
                currentSum -= 11 # 현재 합에서 11 빼고
                currentSum += aceCount # 에이스를 1로 사용해 현재 합에 추가
                break

        return currentSum

    def ShouldHit(self):
        if self.GetValue() >= 17:# 현재 합이 17 이상이면
            return False # 스탠드
        else: # 17 미만이면
            return True # 히트

    def Bust(self): # 버스트 여부 판단
        return self.GetValue() > 21

class StateActionInfo(object):
    # 한 에피소드에서 방문한 (상태, 행동) 쌍을 **중복 없이** 저장
    # → MC 첫방문(First-Visit) 구현에 맞게, 같은 에피소드 내 동일 (s,a)는 한 번만 업데이트
    def __init__(self):
        self.stateActionPairs = [ ]  # 순서 보존 리스트(리턴 후 업데이트용)
        self.stateActionMap = set()  # 중복 체크용 집합

    def AddPair(self, pair):
        # pair: (state, actionBool) 형태
        if pair in self.stateActionMap:
            return  # 이미 기록됐다면 무시(첫방문만 카운트)
        self.stateActionPairs.append(pair)
        self.stateActionMap.add(pair)

def EvaluateAndImprovePolicy(qMap, policy, returns, stateActionPairs, reward):
    """
    에피소드 종료 후, 방문한 모든 (s,a)에 대해:
    - 방문수 N(s,a) 갱신
    - Q(s,a) ← Q(s,a) + (G - Q(s,a)) / N(s,a) (증분 평균 업데이트)
    - 각 상태 s에서 Q(s,True=Hit)와 Q(s,False=Stand)를 비교해 탐욕적으로 정책 개선
    """
    for pair in stateActionPairs:
        # 방문수 1 증가
        returns[pair] += 1
        # 증분 평균: 새 관측값(reward=에피소드 최종 보상)을 이용해 Q 갱신
        qMap[pair] = qMap[pair] + ((reward - qMap[pair]) / returns[pair])

        # 정책 개선: 같은 상태에서 Hit/Stand 중 Q가 큰 쪽을 선택
        state = pair[0]
        shouldHit = False
        if qMap[(state, True)] > qMap[(state, False)]:
            shouldHit = True
        policy[state] = shouldHit

def newCard():
    card = np.random.randint(1, 14) # 카드를 랜덤으로 배정

    if card > 9: # 10 이상의 카드가 나오면
        return 10 # 모두 10으로 처리
    else: # 나머지 카드는
        return card # 숫자 그대로 사용

def PlayEpisode(qMap, policy, returns):
    """
    1개 에피소드를 수행하고, 끝난 뒤 그 에피소드의 결과 보상으로
    방문한 (상태,행동)들에 대해 MC 평가/정책개선을 수행한다.

    - '탐험적 시작(Exploring Starts)': 초기 상태와 첫 행동을 무작위로 선택
      → 모든 (s,a)가 충분히 방문될 기회를 확보
    """
    # --- 탐험적 시작: 무작위 초기 상태 샘플 ---
    playerSum = np.random.randint(11, 22)     # 플레이어 합 11~21
    dealerOpenCard = np.random.randint(1, 11) # 딜러 오픈 카드 1(A)~10
    usableAce = bool(np.random.randint(0, 2)) # 에이스를 11로 쓸 수 있는 상태인지 (True/False)

    player = Player(playerSum, usableAce, dealerOpenCard)
    dealer = Dealer([dealerOpenCard])

    stateActionInfo = StateActionInfo()

    # --- 탐험적 시작: 무작위 '첫 행동' 선택 ---
    hitAction = bool(np.random.randint(0, 2))  # True=Hit, False=Stand
    stateActionInfo.AddPair((player.GetState(), hitAction))

    # --- 플레이어 턴(첫 행동 반영) ---
    if hitAction:
        # 첫 행동이 히트면 카드 1장 받음
        player.ReceiveCard(newCard())

        # 이후에는 버스트가 아니고, 현재 정책이 히트를 권하면 계속 히트
        while not player.Bust() and player.ShouldHit(policy):
            # 방문 (상태,Hit) 기록(첫방문만)
            stateActionInfo.AddPair((player.GetState(), True))
            # 카드 1장 더 받음
            player.ReceiveCard(newCard())

    # --- 플레이어 버스트 체크 ---
    if player.Bust():
        # 플레이어가 버스트면 보상 -1로 에피소드 종료
        EvaluateAndImprovePolicy(qMap, policy, returns,
                                 stateActionInfo.stateActionPairs, -1)
        return

    # --- 스탠드 확정 상태 기록 ---
    # (버스트가 아니라면 결국 스탠드 상태로 넘어가므로, 마지막 상태에 대해 (s,Stand)도 기록)
    stateActionInfo.AddPair((player.GetState(), False))

    # --- 딜러 턴 ---
    # 딜러는 카드 1장 받고(두 번째 카드), 합이 17 미만이면 계속 히트
    dealer.ReceiveCard(newCard())
    while not dealer.Bust() and dealer.ShouldHit():
        dealer.cards.append(newCard())

    # --- 최종 승패/보상 계산 ---
    # 플레이어 승: +1, 패: -1, 무: 0
    if dealer.Bust() or dealer.GetValue() < player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns,
                                 stateActionInfo.stateActionPairs, 1)
    elif dealer.GetValue() > player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns,
                                 stateActionInfo.stateActionPairs, -1)
    else:
        EvaluateAndImprovePolicy(qMap, policy, returns,
                                 stateActionInfo.stateActionPairs, 0)


# =========================
# 테이블/정책 초기화
# =========================
qMap = { }     # Q(s,a) 저장 딕셔너리: key = (state, actionBool)
policy = { }   # 정책 π(s): True=Hit, False=Stand
returns = { }  # 방문수 N(s,a): 증분 평균 계산용

# 상태 공간을 순회하며 Q/N/정책 초기값을 설정
for playerSum in range(11, 22):      # 플레이어 합 11~21
    for usableAce in range(2):       # 0(False), 1(True)
        for dealersCard in range(1, 11):  # 딜러 오픈 1(A)~10
            playerState = (playerSum, bool(usableAce), dealersCard)

            # Q 초기값 0, 방문수 0
            qMap[(playerState, False)] = 0
            qMap[(playerState, True)]  = 0
            returns[(playerState, False)] = 0
            returns[(playerState, True)]  = 0

            # 초기 정책: 20/21은 스탠드, 나머지는 히트
            if playerSum == 20 or playerSum == 21:
                policy[playerState] = False  # Stand
            else:
                policy[playerState] = True   # Hit

# =========================
# MC 학습(에피소드 반복)
# =========================
for i in range(100000):  # 충분히 많은 에피소드로 수렴 유도
    PlayEpisode(qMap, policy, returns)

x11 = [ ]
y11 = [ ]

x12 = [ ]
y12 = [ ]

x21 = [ ]
y21 = [ ]

x22 = [ ]
y22 = [ ]

for playerState in policy:
    if playerState[1]:
        if policy[playerState]:
            x11.append(playerState[2] - 1)
            y11.append(playerState[0] - 11)
        else:
            x12.append(playerState[2] - 1)
            y12.append(playerState[0] - 11)
    else:
        if policy[playerState]:
            x21.append(playerState[2] - 1)
            y21.append(playerState[0] - 11)
        else:
            x22.append(playerState[2] - 1)
            y22.append(playerState[0] - 11)

plt.figure(0)
plt.title('With Usable Ace')
plt.scatter(x11, y11, color='blue')
plt.scatter(x12, y12, color='yellow')
plt.xticks(range(10), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(11), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.figure(1)
plt.title('Without Usable Ace')
plt.scatter(x21, y21, color='blue')
plt.scatter(x22, y22, color='yellow')
plt.xticks(range(10), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(11), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.show()